In [1]:
script_folder = @__DIR__ # folder where this script is located
data_path = joinpath(script_folder, "data") 

using Revise,DelimitedFiles,Random,StaticArrays
using ParallelTemperingMonteCarlo,StructArrays
using BenchmarkTools


Ensemble, potential, params etc

In [2]:
n_atoms = 38

ti = 300.
tf = 1300.
n_traj = 8

temp = TempGrid{n_traj}(ti,tf) 

mc_cycles = 100
mc_sample = 1

displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

evtohartree = 0.0367493
nmtobohr = 18.8973

#parameters taken from L Vocadlo etal J Chem Phys V120N6 2004
n = 8.482
m = 4.692
ϵ = evtohartree*0.0370
a = 0.25*nmtobohr
C = 27.561

pot = EmbeddedAtomPotential(n,m,ϵ,C,a)
#ensemble = NVT(n_atoms)
move_strat = MoveStrategy(ensemble)



pos_cu38 = [[2.3603476948363165, 2.3603476948363165, 0.0],
[6.994369407022418, 2.33998871029911, 0.0],
[2.33998871029911, 6.994369407022418, 0.0],
[-2.3603476948363165, 2.3603476948363165, 0.0],
[-6.994369407022418, 2.33998871029911, 0.0],
[-2.33998871029911, 6.994369407022418, 0.0],
[-2.3603476948363165, -2.3603476948363165, 0.0],
[-6.994369407022418, -2.33998871029911, 0.0],
[-2.33998871029911, -6.994369407022418, 0.0],
[2.3603476948363165, -2.3603476948363165, 0.0],
[6.994369407022418, -2.33998871029911, 0.0],
[2.33998871029911, -6.994369407022418, 0.0],
[0.0, 0.0, 3.3380357219419614],
[4.84532317769689, 0.0, 3.4261608756649893],
[-4.84532317769689, 0.0, 3.4261608756649893],
[0.0, 4.84532317769689, 3.4261608756649893],
[0.0, -4.84532317769689, 3.4261608756649893],
[4.667179058660764, 4.667179058660764, 3.2911441531516483],
[-4.667179058660764, 4.667179058660764, 3.2911441531516483],
[-4.667179058660764, -4.667179058660764, 3.2911441531516483],
[4.667179058660764, -4.667179058660764, 3.2911441531516483],
[0.0, 0.0, -3.3380357219419614],
[4.84532317769689, 0.0, -3.4261608756649893],
[-4.84532317769689, 0.0, -3.4261608756649893],
[0.0, 4.84532317769689, -3.4261608756649893],
[0.0, -4.84532317769689, -3.4261608756649893],
[4.667179058660764, 4.667179058660764, -3.2911441531516483],
[-4.667179058660764, 4.667179058660764, -3.2911441531516483],
[-4.667179058660764, -4.667179058660764, -3.2911441531516483],
[4.667179058660764, -4.667179058660764, -3.2911441531516483], 
[2.327190348361654, 2.327190348361654, 6.600387922922003],
[-2.327190348361654, 2.327190348361654, 6.600387922922003],
[-2.327190348361654, -2.327190348361654, 6.600387922922003],
[2.327190348361654, -2.327190348361654, 6.600387922922003],
[2.327190348361654, 2.327190348361654, -6.600387922922003],
[-2.327190348361654, 2.327190348361654, -6.600387922922003],
[-2.327190348361654, -2.327190348361654, -6.600387922922003],
[2.327190348361654, -2.327190348361654, -6.600387922922003]]
shuffle!(pos_cu38)
n_bin = 100
AtoBohr = 1.8897259886

bc_cu38 = SphericalBC(radius=14*AtoBohr) 
start_config = Config(pos_cu38, bc_cu38)


UndefVarError: UndefVarError: `ensemble` not defined

RunnerPotential

We require a different view for our calculation of tpz and g_offset

In [3]:
X = [ 11              0.001   0.000  11.338
 10              0.001   0.000  11.338
 11              0.020   0.000  11.338
 10              0.020   0.000  11.338
 11              0.035   0.000  11.338
 10              0.035   0.000  11.338
 11              0.100   0.000  11.338
 10              0.100   0.000  11.338
 11              0.400   0.000  11.338
 10              0.400   0.000  11.338]
radsymmvec = []
V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.338],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]
T = [111,110,100]
angularsymmvec = []
#-------------------------------------------#
#-----------Including scaling data----------#
#-------------------------------------------#
file = open(joinpath(data_path,"scaling.data")) # full path "./data/scaling.data"
scalingvalues = readdlm(file)
close(file)
G_value_vec = []
for row in eachrow(scalingvalues[1:88,:])
    max_min = [row[4],row[3]]
    push!(G_value_vec,max_min)
end
for symmindex in eachindex(eachrow(X))
    row = X[symmindex,:]
    radsymm = RadialType2{Float64}(row[2],row[4],Int(row[1]),G_value_vec[symmindex])
    push!(radsymmvec,radsymm)
end
let n_index = 10
for element in V
    for types in T

        n_index += 1

        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types,G_value_vec[n_index])

        push!(angularsymmvec,symmfunc)
    end
end
end

totalsymmvec = vcat(radsymmvec,angularsymmvec)
num_nodes::Vector{Int32} = [88, 20, 20, 1]
activation_functions::Vector{Int32} = [1, 2, 2, 1]
file = open(joinpath(data_path, "weights.029.data"), "r+") # "./data/weights.029.data"
weights=readdlm(file)
close(file)
weights = vec(weights)
nnp = NeuralNetworkPotential(num_nodes,activation_functions,weights)
testpotential = RuNNerPotential(nnp,radsymmvec,angularsymmvec)
#statest,stratest,resultest,nsteptest,startcounttest=initialisation(mc_params,temp,start_config,testpotential,ensemble)

RuNNerPotential{10, 78}(NeuralNetworkPotential(4, 2221, Int32[88, 20, 20, 1], Int32[1, 2, 2, 1], [-1.0768943782, 0.3563458393, -1.3084861447, 0.1127640916, -0.5646591931, -0.4969830793, 1.423770514, -0.2005662393, -0.8957859374, 0.6076110858  …  0.1939524848, -0.0342903191, -0.0231042009, 0.0718481968, -0.0828320122, -0.0728501885, 0.0306628826, 0.0486053813, -0.0867018862, 0.0441651841]), RadialType2{Float64}[RadialType2{Float64}(0.001, 11.338, 11, -0.029892621990594427, 0.06634475788973929), RadialType2{Float64}(0.001, 11.338, 10, -0.0, 0.09828103647521905), RadialType2{Float64}(0.02, 11.338, 11, -0.013430592462258787, 0.13321466604629645), RadialType2{Float64}(0.02, 11.338, 10, -0.0, 0.18618835781674212), RadialType2{Float64}(0.035, 11.338, 11, -0.0067056953127401205, 0.2093521117141925), RadialType2{Float64}(0.035, 11.338, 10, -0.0, 0.28571045376567983), RadialType2{Float64}(0.1, 11.338, 11, -0.00026435144512647774, 0.9364998977117353), RadialType2{Float64}(0.1, 11.338, 10, -0.0, 1

In [4]:
X = [ 2              0.001   0.000  11.338
 #10              0.001   0.000  11.338
 2              0.020   0.000  11.338
 #10              0.020   0.000  11.338
 2              0.035   0.000  11.338
 #10              0.035   0.000  11.338
 2              0.100   0.000  11.338
 #10              0.100   0.000  11.338
 2              0.400   0.000  11.338
 #10              0.400   0.000  11.338
 ]

radsymmvec = []
#--------------------------------------------#
#--------Vector of angular symm values-------#
#--------------------------------------------#
V = [[0.0001,1,1,11.338],
[0.0001,-1,2,11.338],
[0.003,-1,1,11.338],
[0.003,-1,2,11.338],
[0.008,-1,1,11.338],
[0.008,-1,2,11.338],
[0.008,1,2,11.338],
[0.015,1,1,11.338],
[0.015,-1,2,11.338],
[0.015,-1,4,11.338],
[0.015,-1,16,11.338],
[0.025,-1,1,11.338],
[0.025,1,1,11.338],
[0.025,1,2,11.338],
[0.025,-1,4,11.338],
[0.025,-1,16,11.338],
[0.025,1,16,11.338],
[0.045,1,1,11.338],
[0.045,-1,2,11.338],
[0.045,-1,4,11.338],
[0.045,1,4,11.338],
[0.045,1,16,11.338],
[0.08,1,1,11.338],
[0.08,-1,2,11.338],
[0.08,-1,4,11.338],
[0.08,1,4,11.338]]
T = [111,110,100]
angularsymmvec = []
#-------------------------------------------#
#-----------Including scaling data----------#
#-------------------------------------------#
file = open(joinpath(data_path,"scaling.data")) # full path "./data/scaling.data"
scalingvalues = readdlm(file)
close(file)
G_value_vec = []
for row in eachrow(scalingvalues[1:88,:])
    max_min = [row[4],row[3]]
    push!(G_value_vec,max_min)
end
G_value_vec_b = []
for row in eachrow(scalingvalues[89:end,:])
    max_min = [row[4],row[3]]
    push!(G_value_vec_b,max_min)
end
for symmindex in eachindex(eachrow(X))
    row = X[symmindex,:]
    radsymm = RadialType2a{Float64}(row[2],row[4],Int(row[1]),[G_value_vec[(symmindex-1)*2 + 1],G_value_vec[(symmindex-1)*2 + 2]],[G_value_vec_b[(symmindex-1)*2 + 1],G_value_vec_b[(symmindex-1)*2 + 2]])
    push!(radsymmvec,radsymm)
end
let n_index = 10
let j_index = 0
for element in V
    #for types in T

        j_index += 1

        symmfunc = AngularType3a{Float64}(element[1],element[2],element[3],11.338,2,[G_value_vec[n_index + (j_index-1)*3 + 1],G_value_vec[n_index + (j_index-1)*3 + 2],G_value_vec[n_index + (j_index-1)*3 + 3] ],[G_value_vec_b[n_index + (j_index-1)*3 + 1],G_value_vec_b[n_index + (j_index-1)*3 + 2],G_value_vec_b[n_index + (j_index-1)*3 + 3] ])

        push!(angularsymmvec,symmfunc)
    #end
end
end
end
#---------------------------------------------------#
#------concatenating radial and angular values------#
#---------------------------------------------------#
totalsymmvec = vcat(radsymmvec,angularsymmvec)
#--------------------------------------------------#
#-----------Initialising the nnp weights-----------#
#--------------------------------------------------#
num_nodes::Vector{Int32} = [88, 20, 20, 1]
activation_functions::Vector{Int32} = [1, 2, 2, 1]


4-element Vector{Int64}:
 1
 2
 2
 1

In [5]:
file = open(joinpath(data_path, "weights.029.data"), "r+") # "./data/weights.029.data"
weights=readdlm(file)
close(file)
weights = vec(weights)
nnpcu = NeuralNetworkPotential(num_nodes,activation_functions,weights)

file2=open(joinpath(data_path, "weights.030.data"),"r+") #./data/weights.030.data
weights2=readdlm(file2)
close(file2)
weights2 = vec(weights2)
nnpzn= NeuralNetworkPotential(num_nodes,activation_functions,weights2)
ensemble = NNVT([32,6];natomswaps=2)

runnerpotential = RuNNerPotential2Atom(nnpcu,nnpzn,radsymmvec,angularsymmvec,32,6)

RuNNerPotential2Atom{5, 26, 32, 6}(NeuralNetworkPotential(4, 2221, Int32[88, 20, 20, 1], Int32[1, 2, 2, 1], [-1.0768943782, 0.3563458393, -1.3084861447, 0.1127640916, -0.5646591931, -0.4969830793, 1.423770514, -0.2005662393, -0.8957859374, 0.6076110858  …  0.1939524848, -0.0342903191, -0.0231042009, 0.0718481968, -0.0828320122, -0.0728501885, 0.0306628826, 0.0486053813, -0.0867018862, 0.0441651841]), NeuralNetworkPotential(4, 2221, Int32[88, 20, 20, 1], Int32[1, 2, 2, 1], [-0.2990596476, 0.50210302, -0.0769338744, -1.1006272074, 0.2955859359, 1.7429579447, -1.6013368538, 0.2350663304, 0.5352038521, 0.0051334333  …  -0.07023029, -0.0274223406, 0.0379881304, 0.1032826216, -0.0671415566, 0.0873441204, 0.0024981831, -0.1005392202, 0.011513595, -0.0270711056]), RadialType2a{Float64}[RadialType2a{Float64}(0.001, 11.338, 2, [-0.029892621990594427, -0.0, -0.0, -0.0], [0.06634475788973929, 0.09828103647521905, 0.07273034544948624, 0.07540689868304344]), RadialType2a{Float64}(0.02, 11.338, 2, [-

In [6]:
X = ptmc_run!(mc_params,temp,start_config,runnerpotential,ensemble;save=100)

UndefVarError: UndefVarError: `start_config` not defined

In [7]:
isa(runnerpotential,AbstractMachineLearningPotential)

true

In [8]:
runnerpot = RuNNerPotential2Atom(nnpcu,nnpzn,radsymmvec,angularsymmvec,4,2)
v1 = SVector(2.36, 2.36, 0.0)
v2 = SVector(6.99, 2.33, 0.0)
v3 = SVector(2.33, 6.99, 0.0)
v4 = SVector(-2.36, 2.36, 0.0)
v5 = SVector(-6.99, 2.33, 0.0)
v6 = SVector(-2.33, 6.99, 0.0)
bc = SphericalBC(radius=2.0)
conf = Config{6}([v1,v2,v3,v4,v5,v6],bc)
d2mat = get_distance2_mat(conf)
vars = set_variables(conf,d2mat,runnerpot)
evars = set_ensemble_variables(conf,NNVT([4,2]))

NNVTVariables{Float64, 6, 4, 2}(1, [0.0, 0.0, 0.0], [1, 5])

In [9]:
isa(vars.g_matrix,MMatrix)
E,vars = initialise_energy(conf,d2mat,vars,evars,runnerpot)

(-0.2616528993354795, NNPVariables2a{Float64, 6, 88}([-0.07961990236909662, -0.046329508101558725, -0.06652413358790898, -0.08616269292006679, 0.012197910417455157, 0.0047854272256963905], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.09435530394181704 0.040032613593324516 … 0.0516772471800642 0.1181785536901605; 0.041829635204485544 0.0014182473523196017 … 0.02698557697421576 0.02698557697421576; … ; 0.012001828644531058 4.708405159025907e-7 … 0.007679720869687302 0.007679027976057983; 7.081025391239545e-7 0.0 … 0.0 0.0], [1.0 0.6419526956120072 … 0.07395536545379516 0.3737328615585688; 0.6419526956120072 1.0 … 0.0 0.016085593143803867; … ; 0.07395536545379516 0.0 … 1.0 0.37375116838713807; 0.3737328615585688 0.016085593143803867 … 0.37375116838713807 1.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))

In [10]:
E ≈-0.261652899

true